# GAN生成器网络

In [2]:
import keras
from keras import layers
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

latent_dim = 32 # 随机噪声的维度
height, width, channels = 32, 32, 3 # 生成图片的尺寸

generator_input = keras.Input(shape=(laten_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
# 生成器Model实例化 讲形状为(laten_dim,)的张量映射为形状为(height, width, channels)的图像
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_1 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32768)             0         
                                                                 
 reshape_1 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      1048

# GAN判别器网络

In [3]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides= 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.3)(x) # Dropout层有助于减少过拟合

x = layers.Dense(1, activation='sigmoid')(x)
# 判别器Model实例化 讲形状为(height, width, channels)的图像映射为0到1之间的实数
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    learning_rate=0.0008,
    clipvalue=1.0,
    decay=1e-8 # 限制梯度值的范围 进行梯度剪枝
)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 30, 30, 128)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 6, 6, 128)         0   

# 对抗网络

In [4]:
discriminator.trainable = False # 判别器在对抗网络中是不可训练的
gan_input = keras.Input(shape= (latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# 训练DCGAN

In [6]:
import os
import tensorflow
from keras.preprocessing import image
# 加载数据集
(x_train, y_train), (_,_) = tensorflow.keras.datasets.cifar10.load_data()
x_train = x_train.reshape((x_train.shape[0],) + (32, 32, 3)).astype('float32') / 255.0 # 数据标准化
iterations = 10000 # 迭代次数
batch_size = 20
save_dir = r'C:\Users\Administrator\PycharmProjects\深度学习日常练习项目\python深度学习红书\生成式深度学习\生成式对抗网络\GAN网络生成图片'

start = 0
for step in range(iterations):
    # 在潜在空间随机采集点
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    # 通过生成器生成图像
    generated_imags = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop] # 从真实数据集中采集图像
    combined_images = np.concatenate([generated_imags, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))]) # 生成标签 1表示真实图像 0表示生成图像 以便区分
    labels += 0.05 * np.random.random(labels.shape) # 添加噪声
    d_loss = discriminator.train_on_batch(combined_images, labels) # 训练判别器
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim)) # 在潜在空间随机采集点
    misleading_targets = np.zeros((batch_size, 1)) # 生成标签 1表示真实图像 0表示生成图像 以便区分
    # 通过gan模型训练生成器 目标是让生成器愚弄判别器 让判别器认为生成的图像是真实的
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets) # 训练生成器
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    # 每100步保存一次生成的图像和模型权重
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        # 保存生成的图像
        print('step %d: d_loss = %f, a_loss = %f' % (step, d_loss, a_loss))
        img = image.array_to_img(generated_imags[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_fantasy.png'))
        img = image.array_to_img(real_images[0] * 255., scale=False)# 保存真实图像
        img.save(os.path.join(save_dir, 'real_fantasy.png'))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\.keras\\datasets\\cifar-10-batches-py\\data_batch_1'